# TELCO CUSTOMER CHURN PREDICTION


### Project Overview

The primary objective of this project is to develop a predictive model that accurately identifies customers who are likely to churn in the near future. By leveraging historical data, the model aims to help the telecommunications company proactively retain at-risk customers through targeted interventions and personalized offers.

I will be leveraging the widely used CRISP-DM Framework for this project.

The CRISP-DM (Cross-Industry Standard Process for Data Mining) framework is a widely used methodology for data mining and data analysis projects. It provides a structured approach for planning and executing data projects. The process is cyclic and iterative, consisting of six phases:

1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

## Business Understanding


### Hypotheses
- Hypothesis 1:

Assumption : Customers with longer tenure are less likely to churn.

Rationale behind assumption : Long-term customers may have higher satisfaction and loyalty due to their prolonged association with the company and potential benefits received over time.


- Hypothesis 2:

Assumption : Customers who use multiple services (e.g., Internet, phone, TV) are less likely to churn compared to those who use fewer services.

Rationale behind assumption : Customers with bundled services may find it more convenient and cost-effective to stay with the provider, making them less likely to switch.


- Hypothesis 3:

Assumption : Customers with higher monthly charges are more likely to churn.

Rationale behind assumption : Higher monthly charges might indicate dissatisfaction with the value for money, leading customers to seek cheaper alternatives.


### Important Business Questions

1. What are the main reasons behind customer churn?

   - Understanding the primary factors contributing to churn can help in addressing the root causes and developing effective retention strategies.


2. Which customer demographics (e.g., age, gender, marital status) are more likely to churn?

   - Identifying specific demographic groups that are more prone to churn can help tailor marketing and retention efforts to those segments.


3. How does the length of the customer's tenure impact their likelihood to churn?

   - Analyzing the relationship between tenure and churn can reveal whether long-term customers are more loyal and what measures can be taken to extend customer tenure.


4. What role do service issues (e.g., technical support, service outages) play in customer churn?

   - Assessing the impact of service quality on churn can guide improvements in customer support and service reliability.


5. How do different payment methods (e.g., automatic payments vs. manual payments) affect churn rates?

   - Understanding the influence of payment methods on churn can help in promoting more stable payment options that reduce churn.


6. What is the impact of contract types (e.g., month-to-month vs. annual contracts) on customer retention?

   - Analyzing how contract terms affect churn can inform decisions on offering more flexible or beneficial contract options to customers.


7. Which services or combinations of services (e.g., Internet, phone, TV) have the highest and lowest churn rates?

   - Identifying which services are most and least likely to retain customers can help in designing better service packages and cross-selling strategies.

In [23]:
import pyodbc #just installed with pip
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

Then I will go ahead to get my credentials needed to connect to the database, from my .env file.

In [24]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


# Get the values for the credentials you set in the '.env' file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

#create connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

In [26]:
connection = pyodbc.connect(connection_string)

## Data Understanding

The data for this projects has been divided into 3. The first 2 data sets are for training and evaluation the machine learning model  while the last data set is for testing the model.

The first 3000 records of the dataset can be found in a database I will be connecting to using some credentials.

The second part of the data is hosted on a GitHub Repository in a file called LP2_Telco-churn-second-2000.csv.

The final 2000 records of the data set needed for this project can be found in a OneDrive. The filed is named Telco-churn-last-2000.xlsx. 

Below are the columns and their descriptions:

| Attribute          | Description                                                    |
|--------------------|----------------------------------------------------------------|
| Gender             | Whether the customer is a male or a female                     |
| SeniorCitizen      | Whether a customer is a senior citizen or not                  |
| Partner            | Whether the customer has a partner or not (Yes, No)            |
| Dependents         | Whether the customer has dependents or not (Yes, No)           |
| Tenure             | Number of months the customer has stayed with the company      |
| PhoneService       | Whether the customer has a phone service or not (Yes, No)      |
| MultipleLines      | Whether the customer has multiple lines or not                 |
| InternetService    | Customer's internet service provider (DSL, Fiber Optic, No)    |
| OnlineSecurity     | Whether the customer has online security or not (Yes, No, No Internet) |
| OnlineBackup       | Whether the customer has online backup or not (Yes, No, No Internet) |
| DeviceProtection   | Whether the customer has device protection or not (Yes, No, No internet service) |
| TechSupport        | Whether the customer has tech support or not (Yes, No, No internet) |
| StreamingTV        | Whether the customer has streaming TV or not (Yes, No, No internet service) |
| StreamingMovies    | Whether the customer has streaming movies or not (Yes, No, No Internet service) |
| Contract           | The contract term of the customer (Month-to-Month, One year, Two year) |
| PaperlessBilling   | Whether the customer has paperless billing or not (Yes, No)    |
| PaymentMethod      | The customer's payment method (Electronic check, mailed check, Bank transfer(automatic), Credit card(automatic)) |
| MonthlyCharges     | The amount charged to the customer monthly                     |
| TotalCharges       | The total amount charged to the customer                       |
| Churn              | Whether the customer churned or not (Yes or No)                |


In [28]:
# query the database for the first part of the data and assign it a variable name 'df1'
query = "Select * from dbo.LP2_Telco_churn_first_3000"

df1 = pd.read_sql(query, connection)

In [29]:
df1.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,...,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,...,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,...,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,...,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,...,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True


In [31]:

# read in the second part of the data from github and assign it a variable name 'df2'
git_raw_url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_LP2-Classifcation/main/LP2_Telco-churn-second-2000.csv'

df2 = pd.read_csv(git_raw_url)
df2.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5600-PDUJF,Male,0,No,No,6,Yes,No,DSL,No,...,No,Yes,No,No,Month-to-month,Yes,Credit card (automatic),49.50,312.7,No
1,8292-TYSPY,Male,0,No,No,19,Yes,No,DSL,No,...,Yes,Yes,No,No,Month-to-month,Yes,Credit card (automatic),55.00,1046.5,Yes
2,0567-XRHCU,Female,0,Yes,Yes,69,No,No phone service,DSL,Yes,...,Yes,No,No,Yes,Two year,Yes,Credit card (automatic),43.95,2960.1,No
3,1867-BDVFH,Male,0,Yes,Yes,11,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,74.35,834.2,Yes
4,2067-QYTCF,Female,0,Yes,No,64,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,111.15,6953.4,No
